In [43]:
import os
import random
import json
from PIL import Image

In [44]:
# 데이터 경로 설정
base_path = "./dataset/Validation"
output_base_path = "./crop_dataset/Validation"  # 크롭된 이미지 저장 폴더

In [45]:
# 크롭된 이미지를 저장할 폴더가 없으면 생성
if not os.path.exists(output_base_path):
    os.makedirs(output_base_path)

In [46]:
# 작물 목록
crops = ['고추', '무', '배추', '오이', '파']

### 바운딩 박스 라벨로 이미지 크롭
- 바운딩 박스의 중심을 기준으로 정사각형으로 이미지를 자르기
- 자른 이미지는 따로 저장

In [ ]:
def crop_square_image(image_path, label_path):
    try:
        # 이미지 로드
        image = Image.open(image_path)
        image_width, image_height = image.size

        # 라벨 로드
        with open(label_path, 'r', encoding='utf-8') as f:
            label_data = json.load(f)
        
        # 바운딩 박스 정보 추출
        annotations = label_data.get("annotations", {})
        points = annotations.get("points", [])
        
        if not points:
            print(f"No bounding box found for {os.path.basename(image_path)}")
            return None  # 바운딩 박스가 없으면 None 반환

        # 바운딩 박스 중심과 길이 계산
        point = points[0]  # 첫 번째 바운딩 박스만 처리
        xtl, ytl, xbr, ybr = point['xtl'], point['ytl'], point['xbr'], point['ybr']
        
        # 바운딩 박스 중심 좌표
        center_x = int((xtl + xbr) / 2)
        center_y = int((ytl + ybr) / 2)
        
        # 바운딩 박스 너비와 높이
        bbox_width = xbr - xtl
        bbox_height = ybr - ytl

        # 정사각형 한 변의 길이 (가로/세로 중 더 긴 값)
        square_side = max(bbox_width, bbox_height)
        
        # 정사각형 영역 계산
        left = max(center_x - square_side // 2, 0)
        top = max(center_y - square_side // 2, 0)
        right = min(center_x + square_side // 2, image_width)
        bottom = min(center_y + square_side // 2, image_height)
        
        # 좌표 검증
        if left >= right or top >= bottom:
            print(f"Invalid crop coordinates for {os.path.basename(image_path)}: "
                  f"left={left}, top={top}, right={right}, bottom={bottom}")
            return None
        
        # 이미지 자르기
        cropped_image = image.crop((left, top, right, bottom))
        return cropped_image

    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

In [48]:
# 크롭 후 이미지를 저장하는 함수
def crop_and_save_images():
    for crop in crops:
        # 원천 이미지 폴더와 라벨 폴더 경로 설정
        original_folder = os.path.join(base_path, f"[원천]{crop}_1.질병")
        label_folder = os.path.join(base_path, f"[라벨]{crop}_1.질병")
        
        # 크롭된 이미지를 저장할 폴더 설정
        output_folder = os.path.join(output_base_path, crop+"_1.질병")
        
        # 크롭된 이미지 폴더가 없으면 생성
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        # 원천 데이터에서 파일 선택 (대소문자 구분 없이 확장자 처리)
        original_files = [f for f in os.listdir(original_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        
        # 라벨 파일과 이미지를 순회하면서 처리
        for file_name in original_files:
            image_path = os.path.join(original_folder, file_name)
            label_path = os.path.join(label_folder, file_name + '.json')
            
            # 이미지 크롭
            cropped_image = crop_square_image(image_path, label_path)
            
            if cropped_image:
                # 저장할 파일 이름
                save_path = os.path.join(output_folder, f"cropped_{file_name}")
                cropped_image.save(save_path)
                print(f"Saved cropped image: {save_path}")

In [49]:
# 실행
crop_and_save_images()

Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_1_0424z_20200929_6.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_1_0424z_20201023_11.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_1_0868z_20201019_13.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_1_0868z_20201026_3.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_1_1553bb_20200928_23.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_1_1553bb_20201008_19.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_1_5430z_20200921_4.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_2_0149z_20201007_16.JPG
Saved cropped image: ./crop_dataset/Validation\고추_1.질병\cropped_V006_79_1_01_01_01_13_2_0149z_20201007_21.JPG
Saved cropped image: